In [34]:
import pandas as pd

# Load files
metrics = pd.read_csv("daily_metrics.csv")
sentiment = pd.read_csv("fear_greedupdated.csv")

# Convert both to datetime
metrics['Date'] = pd.to_datetime(metrics['Date'])
sentiment['date'] = pd.to_datetime(sentiment['date'], dayfirst=True)

# Merge

merged = metrics.merge(sentiment[['date','classification']], 
                       left_on='Date', right_on='date', how='left')
# Rename for clarity
merged = merged.rename(columns={'classification':'Sentiment'})
performance = merged.groupby('Sentiment')[['Closed PnL','Win Rate','Leverage']].mean()
print(performance)

#Trade freq
trade_freq = merged.groupby('Sentiment')['Side'].count()
#position freq
position_sizes = merged.groupby('Sentiment')[['Size USD','Size Tokens']].mean()
#bias freq
bias = merged.groupby('Sentiment')['LongShortRatio'].mean()
#leverage freq
leverage_dist = merged.groupby('Sentiment')['Leverage'].describe()

trader_stats = merged.groupby('Account').agg({'Leverage':'mean','Closed PnL':'sum','Win Rate':'mean','Side':'count'}).reset_index()

# High vs Low Leverage

median_leverage = trader_stats['Leverage'].median()
high_leverage = trader_stats[trader_stats['Leverage'] > median_leverage]
low_leverage = trader_stats[trader_stats['Leverage'] <= median_leverage]

# Frequent vs Infrequent

median_trades = trader_stats['Side'].median()
frequent = trader_stats[trader_stats['Side'] > median_trades]
infrequent = trader_stats[trader_stats['Side'] <= median_trades]

# Consistent vs Inconsistent

consistent = trader_stats[trader_stats['Win Rate'] > 0.6]
inconsistent = trader_stats[trader_stats['Win Rate'] <= 0.6]


#for chart and plots
import seaborn as sns
import matplotlib.pyplot as plt
# PnL by sentiment
sns.barplot(data=merged, x='Sentiment', y='Closed PnL')
plt.title("PnL by Sentiment")
plt.close()

# Leverage distribution
sns.boxplot(data=merged, x='Sentiment', y='Leverage')
plt.title("Leverage Distribution by Sentiment")
plt.close()

# Long/Short ratio
sns.barplot(data=merged, x='Sentiment', y='LongShortRatio')
plt.title("Long/Short Ratio by Sentiment")
plt.close()

                Closed PnL  Win Rate      Leverage
Sentiment                                         
Extreme Fear   4619.439053  0.329659  7.939846e+04
Extreme Greed  5161.922644  0.386387  2.020237e+04
Fear           5328.818161  0.364033  1.142951e+07
Greed          3318.100730  0.343559  2.779431e+07
Neutral        3438.618818  0.355414  1.163047e+07
